### Part 1:

Seems like a Riddler problem to me. Building a class that tracks some basic information:
- position of player on board
- board size
- score
- score list (because I kept messing up my modulus to determine location on board...oops)

Also has one method, `move`:
- move will take the rolled amount and add to current position, taking the modulo
- i kept messing this up (10%10 == 0, not 10) so i just use this as an index on the initial score list, which nicely matches the board

Another cool tool, itertool's `cycle` object:
- it is a generator object that resets itself once it hits the typical `StopIteration`, which is perfect for this problem

In [1]:
from itertools import cycle, product
from collections import defaultdict
        
class player():
    def __init__(self, start, board_size = 10):
        self.position = start
        self.board_size = board_size
        self.score = 0 # init to 0
        self.score_list = [1,2,3,4,5,6,7,8,9,10]
    
    def move(self, roll):
        # this ugly
        self.position = self.score_list[(self.position + roll - 1) % self.board_size]
        self.score += self.position
        
# minimal test
b = player(8, 10)
rolls = 4+5+6
b.move(rolls)
assert(b.position == 3)
assert(b.score == 3)
rolls = 10+11+12
b.move(rolls)
assert(b.position == 6)
assert(b.score == 9)

In [2]:
# Actual Input
# start with our deterministic dice, using the awesome "cycle"
dd = cycle(range(1,101))

a = player(8, 10)
b = player(2, 10)

roll_count = 0 # when odd we apply to a otherwise b

while True:
    
    # do 3 rolls and sum
    rolls = next(dd) + next(dd) + next(dd)
    roll_count += 1
    
    # apply roll to proper player
    if roll_count % 2 == 1:
        a.move(rolls)        
        if a.score >= 1000:
            loser_score = b.score
            break
    else:
        b.move(rolls)
        if b.score >= 1000:
            loser_score = a.score  
            break
    
print(f"Answer: {loser_score * roll_count * 3}")

Answer: 513936


### Part 2:

Our problem expands as follows:
- We now have a quantum 3-sided dice (values 1,2,3)
- each roll splits off into its own universe -> one where the roll is 1, one where it was 2, and one where it was 3.
    - If we do the math (and I failed to initially!) this means each sequence of 3 rolls can yield 3 x 3 x 3 combinations, or 27 total variants.
    - Some of these variants will repeat in sum (min is 3, max is 9)
- The game plays the same as before, except we now only play until someone hits 21

In how many universes does the player with more wins win out?

#### Problem Approach

Initially I started solving by keeping track of likelihoods of each state, where a state was the following: `{p1 position, p1 score, p2 position, and p2 score}`. 
- I quickly moved away from tracking likelihoods to just keeping counts. 
- The reason for this is I was converting likelihoods into frequency distributions anyways, so no point in worrying about conversion based on expected universes. 

I have a function that takes in the current `state` of a game as well as the `count`.
- The count is how many of these states exist across all universes - I keep tabs on this by using a `defaultdict` and making the `key` the `state`. 
- Depending on which player rolled, each initial state will output 27 states (all combinations of rolls), updating the `position` and `score` of the player that rolled.
- These states have a `count`, which will be added back to the general dictionary keeping tabs on states.m

At the end of a step, I check the state dictionary to see if any of the keys indicate a player has hit or exceeded 21. In the event that they do, I remove this from future steps and add to the players overall winnings. 

In [3]:
# gather possible states
roll_state = [1,2,3]
roll_states = [sum(rolls) for rolls in product(roll_state, roll_state, roll_state)]
score_list = [1,2,3,4,5,6,7,8,9,10]


def quantumSplitCount(p1_loc, p1_score, count, p2_loc, p2_score, roller):
    """
    This might be the gnarliest list comprehension I have ever written
    
    Access current state of universe (p1_loc, p1_score, count, p2_loc, p2_score)
    Depending on the player, built a list representing all possible states based on
    all available rolls and scores. 
    
    Note: Count will be equal across the board since we have that many of the initial state.
    
    Return state list
    
    """
    global score_list
    global roll_states
    
    if roller == 1:
        return [(score_list[(p1_loc + roll - 1) % 10], 
                 p1_score + score_list[(p1_loc + roll - 1) % 10], 
                 count,
                 p2_loc,
                 p2_score
                ) for roll in roll_states]
    else:
        return [(p1_loc, 
                 p1_score, 
                 count,
                 score_list[(p2_loc + roll - 1) % 10],
                 p2_score + score_list[(p2_loc + roll - 1) % 10]
                ) for roll in roll_states]

In [4]:
# Starting info for p1 and p2
p1_loc, p1_score, p1_wins = (4,0,0) 
p2_loc, p2_score, p2_wins = (8,0,0) 

# Store state of all universes
state_dict = defaultdict(lambda: 0)

# Add the origin of universes
state_dict[(p1_loc, p1_score, p2_loc, p2_score)] = 1
step = 0

# Split universes until all universes have a winner
while True:
    step += 1
    player = step % 2 # determine player
    next_state = defaultdict(lambda: 0) # store current run

    # Run quantum split
    for k,count in state_dict.items():
        p1,s1,p2,s2 = k
        output_states = quantumSplitCount(p1,s1,count, p2,s2,player)
        
        # Update for next state, key is state and val is count of universes of this state
        for p1,s1,c, p2,s2 in output_states:
            next_state[(p1,s1, p2,s2)] += c

    # This is not super clean, but wiping most recent state and storing over
    state_dict = defaultdict(lambda: 0)
    for k,count in next_state.items():
        p1,s1,p2,s2 = k
        if s1 >= 21:
            p1_wins += count
        elif s2 >= 21:
            p2_wins += count
        else:
            state_dict[k] += count
    
    # Expansion stops
    if len(state_dict) == 0:
        print(f"Finished after {step} steps")
        print(f"P1 winners: {p1_wins}")
        print(f"P2 winners: {p2_wins}")
        assert(p1_wins == 444356092776315)
        assert(p2_wins == 341960390180808)

        break

Finished after 19 steps
P1 winners: 444356092776315
P2 winners: 341960390180808


In [5]:
import time 

# Starting info for p1 and p2
p1_loc, p1_score, p1_wins = (8,0,0) 
p2_loc, p2_score, p2_wins = (2,0,0) 

# Store state of all universes
state_dict = defaultdict(lambda: 0)

# Add the origin of universes
state_dict[(p1_loc, p1_score, p2_loc, p2_score)] = 1
step = 0

start = time.time()

# Split universes until all universes have a winner
while True:
    step += 1
    player = step % 2 # determine player
    next_state = defaultdict(lambda: 0) # store current run

    # Run quantum split
    for k,count in state_dict.items():
        p1,s1,p2,s2 = k
        output_states = quantumSplitCount(p1,s1,count, p2,s2,player)
        
        # Update for next state, key is state and val is count of universes of this state
        for p1,s1,c, p2,s2 in output_states:
            next_state[(p1,s1, p2,s2)] += c

    # This is not super clean, but wiping most recent state and storing over
    state_dict = defaultdict(lambda: 0)
    for k,count in next_state.items():
        p1,s1,p2,s2 = k
        if s1 >= 21:
            p1_wins += count
        elif s2 >= 21:
            p2_wins += count
        else:
            state_dict[k] += count
    
    # Expansion stops
    if len(state_dict) == 0:
        print(f"Finished after {step} steps")
        print(f"Total time: {time.time() - start:.2f}")
        print(f"P1 winners: {p1_wins}")
        print(f"P2 winners: {p2_wins}")
        break

Finished after 19 steps
Total time: 0.55
P1 winners: 105619718613031
P2 winners: 94052321632284
